### Importing Libraries

In [1]:
import json
import pandas as pd

### Loading Data

Data Source : [layoffs.fyi](https://layoffs.fyi/)

In [2]:
with open('layoff_2023.txt') as f:
    json_data = json.load(f)

### Data Preprocessing

In [3]:
key_map = {}
            
for item in json_data['data']['table']['columns']:
    id_ = item['id']
    name_ = item['name']
    key_map[id_] = name_
    
#Location HQ
for item in json_data['data']['table']['columns']:
    if item['id']=='fld1mbRJlknicV1Dk':
        for i in item['typeOptions']['choices'].values():
            id_ = i['id']
            name_ = i['name']
            key_map[id_] = name_
    
#Industry
for item in json_data['data']['table']['columns']:
    if item['id']=='fldMvtAoxHCri39uZ':
        for i in item['typeOptions']['choices'].values():
            id_ = i['id']
            name_ = i['name']
            key_map[id_] = name_

#Country
for item in json_data['data']['table']['columns']:
    if item['id']=='fldnR6AcR7HjRtwrG':
        for i in item['typeOptions']['choices'].values():
            id_ = i['id']
            name_ = i['name']
            key_map[id_] = name_

#Stage
for item in json_data['data']['table']['columns']:
    if item['id']=='fldbWCltYdMzywX2v':
        for i in item['typeOptions']['choices'].values():
            id_ = i['id']
            name_ = i['name']
            key_map[id_] = name_

In [4]:
row_data = []
for item in json_data['data']['table']['rows']:
    item['cellValuesByColumnId']['id'] = item['id']
    row_data.append(item['cellValuesByColumnId'])

In [5]:
def replace_values(data, replacements):
    for item in data:
        for key, value in item.items():
            if key in ['fld1mbRJlknicV1Dk','fldMvtAoxHCri39uZ','fldbWCltYdMzywX2v','fldnR6AcR7HjRtwrG']:
                if isinstance(value, list):
                    item[key] = [replacements[item] for item in value]
                else:
                    item[key] = replacements[value]

replace_values(row_data, key_map)

In [6]:
def replace_keys(data, replacements):
    for item in data:
        item_copy = item.copy()
        for key, value in item_copy.items():
            item[replacements.get(key, key)] = item.pop(key, None)

replace_keys(row_data, key_map)

In [7]:
dataframe = pd.DataFrame(row_data)
dataframe['Date'] = pd.to_datetime(dataframe['Date']).dt.date
dataframe['Date Added'] = pd.to_datetime(dataframe['Date Added']).dt.date
dataframe.head()

,Company,Location HQ,Industry,# Laid Off,Date,Source,$ Raised (mm),Stage,Date Added,Country,id,%,List of Employees Laid Off
0,Uber,[SF Bay Area],Transportation,200.0,2023-06-21,https://www.nasdaq.com/articles/uber-to-lay-of...,25200.0,Post-IPO,2023-06-22,United States,recnjK8075T9xnImr,NaN,NaN
1,Friday Health Plans,[Alamosa],Healthcare,NaN,2023-06-21,https://www.bizjournals.com/denver/news/2023/0...,306.0,Private Equity,2023-06-22,United States,recC4vnQrPlWFcYXN,NaN,NaN
2,Grab,"[Singapore, Non-U.S.]",Transportation,1000.0,2023-06-20,https://www.reuters.com/technology/singapores-...,16500.0,Post-IPO,2023-06-20,Singapore,recymfZ42XIc7nPPP,0.11,NaN
3,OLX Group,"[Amsterdam, Non-U.S.]",Marketing,800.0,2023-06-20,https://techcrunch.com/2023/06/20/olx-group-la...,NaN,Acquired,2023-06-20,Netherlands,rec7Y1yuHIiivUZOy,NaN,NaN
4,Karakuki,"[London, Non-U.S.]",Food,NaN,2023-06-20,https://www.therobotreport.com/food-robotics-s...,13.0,Unknown,2023-06-22,United Kingdom,recc3HT6NsYGECgEk,1.00,NaN


In [8]:
from datetime import date

data_dict = dataframe.to_dict(orient='records')

data_dict_serializable = [{k: v.strftime('%Y-%m-%d') if isinstance(v, date) else v for k, v in row.items()} for row in data_dict]

with open('Layoffs - Preprocessed.json', 'w') as json_file:
    json.dump(data_dict_serializable, json_file)